In [1]:
## load myqwen model ,tokenizer, processor and upload to hf
import os
import sys
import huggingface_hub
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
saved_path = 'myqwen'
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    saved_path,
    torch_dtype="auto",
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(saved_path)
tokenizer = AutoTokenizer.from_pretrained(saved_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
processor.push_to_hub("mini4o", use_auth_token=True, commit_message="upload mini4o processor")
tokenizer.push_to_hub("mini4o", use_auth_token=True, commit_message="upload mini4o tokenizer")

In [2]:
model.push_to_hub("mini4o", use_auth_token=True, commit_message="upload myqwen model")

C:\Users\29058\Desktop\minigpt-4o\transformers\src\transformers\utils\hub.py:903: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
C:\Users\29058\Desktop\minigpt-4o\transformers\src\transformers\modeling_utils.py:3389: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.51G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/magician-blue/mini4o/commit/887b59eb72ed5fe31ca29c234df411531a79327f', commit_message='upload myqwen model', commit_description='', oid='887b59eb72ed5fe31ca29c234df411531a79327f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/magician-blue/mini4o', endpoint='https://huggingface.co', repo_type='model', repo_id='magician-blue/mini4o'), pr_revision=None, pr_num=None)

In [13]:
!huggingface-cli login

: 

In [1]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct",
    torch_dtype="auto",
    device_map="auto",
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [1]:
from mini4o.processor import Mini4oProcessor
processor = Mini4oProcessor.from_pretrained("myqwen")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
from transformers import AutoTokenizer, AutoProcessor, Qwen2_5_VLForConditionalGeneration
import torch
from PIL import Image


# 加载 tokenizer 和 processor
tokenizer = AutoTokenizer.from_pretrained("myqwen")
# processor = AutoProcessor.from_pretrained("myqwen")
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
            {
                "type": "image_gen",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {
                "type": "image_gen",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
        ],
    }
]


from qwen_vl_utils import process_vision_info, extract_vision_info, fetch_image, fetch_video

def my_process_vision_info(
    conversations: list[dict] | list[list[dict]],
) -> tuple[list[Image.Image] | None, list[torch.Tensor | list[Image.Image]] | None]:
    vision_infos = extract_vision_info(conversations)
    ## Read images or videos
    image_inputs = []
    video_inputs = []
    image_gen_inputs = []
    video_gen_inputs = []
    for vision_info in vision_infos:
        if "image" in vision_info or "image_url" in vision_info:
            if vision_info['type'] == "image_gen":
                image_gen_inputs.append(fetch_image(vision_info))
            else:
                image_inputs.append(fetch_image(vision_info))
        elif "video" in vision_info:
            if vision_info['type'] == "video_gen":
                video_gen_inputs.append(fetch_video(vision_info))
            else:
                video_inputs.append(fetch_video(vision_info))
        else:
            raise ValueError("image, image_url or video should in content.")
    if len(image_inputs) == 0:
        image_inputs = None
    if len(video_inputs) == 0:
        video_inputs = None
    if len(image_gen_inputs) == 0:
        image_gen_inputs = None
    if len(video_gen_inputs) == 0:
        video_gen_inputs = None
    return image_inputs, video_inputs, image_gen_inputs, video_gen_inputs


# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
print(text)
image_inputs, video_inputs, image_gen_inputs, video_gen_inputs = my_process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    images_gen=image_gen_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
# inputs = inputs.to("cuda")

# # Inference: Generation of the output
# generated_ids = model.generate(**inputs, max_new_tokens=128)
# generated_ids_trimmed = [
#     out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
# ]
# output_text = processor.batch_decode(
#     generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
# )
# print(output_text)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>
Describe this image.<|image_gen_start|><|image_gen_pad|><|image_gen_end|>
<|vision_start|><|image_pad|><|vision_end|>
<|image_gen_start|><|image_gen_pad|><|image_gen_end|>
<|im_end|>
<|im_start|>assistant



In [17]:
x = processor.batch_decode(inputs.input_ids, skip_special_tokens=False, clean_up_tokenization_spaces=False)[0]
print(x)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
            {
                "type": "image_gen",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {
                "type": "image_gen",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
        ],
    }
]

In [9]:
x.count("<|image_gen_pad|>")

512

In [19]:
from transformers import AutoTokenizer, AutoProcessor, Qwen2_5_VLForConditionalGeneration

# 加载 tokenizer 和 processor
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

# 定义你要加的 special tokens
special_tokens = [
    "<|image_gen_start|>", "<|image_gen_pad|>", "<|image_gen_end|>",
    "<|video_gen_start|>", "<|video_gen_pad|>", "<|video_gen_end|>"
]

# 添加 token，并获得它们的 ID
tokenizer.add_special_tokens({"additional_special_tokens": special_tokens+tokenizer.additional_special_tokens})
processor.tokenizer = tokenizer  # 更新 processor 的 tokenizer

# 映射到 ID
token_ids = {token: tokenizer.convert_tokens_to_ids(token) for token in special_tokens}


In [ ]:
model.resize_token_embeddings(len(tokenizer))
# 显式写入 config
model.config.image_gen_start_token_id = token_ids["<|image_gen_start|>"]
model.config.image_gen_pad_token_id   = token_ids["<|image_gen_pad|>"]
model.config.image_gen_end_token_id   = token_ids["<|image_gen_end|>"]
model.config.video_gen_start_token_id = token_ids["<|video_gen_start|>"]
model.config.video_gen_pad_token_id   = token_ids["<|video_gen_pad|>"]
model.config.video_gen_end_token_id   = token_ids["<|video_gen_end|>"]

# 也可以存一个列表，如果你喜欢
model.config.additional_special_tokens = special_tokens
model.config.additional_special_tokens = None

In [11]:
# 去掉model.config.additional_special_tokens
model.config.num_image_gen_tokens = 256

In [12]:
# https://huggingface.co/Qwen/Qwen2.5-VL-3B-Instruct
# Sana 0.
save_path = "myqwen"

tokenizer.save_pretrained(save_path)
processor.save_pretrained(save_path)
model.save_pretrained(save_path)
model.config.save_pretrained(save_path)


C:\Users\29058\Desktop\minigpt-4o\transformers\src\transformers\modeling_utils.py:3389: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
save_path = "myqwen"

tokenizer.save_pretrained(save_path)

('myqwen\\tokenizer_config.json',
 'myqwen\\special_tokens_map.json',
 'myqwen\\chat_template.jinja',
 'myqwen\\vocab.json',
 'myqwen\\merges.txt',
 'myqwen\\added_tokens.json',
 'myqwen\\tokenizer.json')

In [8]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained('myqwen')

print(config.image_gen_start_token_id)  # 应该输出你加的 token 的 ID

151665


In [1]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
# default: Load the model on the available device(s)
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto"
# )

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct",
    torch_dtype=torch.bfloat16,
    attn_implementation="sdpa",
    device_map="auto",
)

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [10]:
from qwen_vl_utils import process_vision_info
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

['The image depicts a serene beach scene with a person and a dog. The person is sitting on the sandy beach, facing the ocean, and appears to be interacting with the dog. The dog is also sitting on the sand, facing the person, and seems to be giving a paw or a friendly gesture. The person is wearing a plaid shirt and has long hair. The background shows the ocean with gentle waves and a clear sky, suggesting it might be early morning or late afternoon due to the soft lighting. The overall atmosphere of the image is peaceful and joyful.']


In [9]:
print(text)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>Describe this image.<|vision_start|><|image_pad|><|vision_end|><|im_end|>
<|im_start|>assistant



In [13]:
print(processor.batch_decode(inputs.input_ids, skip_special_tokens=False, clean_up_tokenization_spaces=False)[0])

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|

In [1]:
# run `pip install git+https://github.com/huggingface/diffusers` before use Sana in diffusers
import torch
from diffusers import SanaPipeline

pipe = SanaPipeline.from_pretrained(
    "Efficient-Large-Model/Sana_Sprint_0.6B_1024px_diffusers",
    torch_dtype=torch.bfloat16,
)
pipe.to("cuda")

pipe.vae.to(torch.bfloat16)
pipe.text_encoder.to(torch.bfloat16)

prompt = 'a cyberpunk cat with a neon sign that says "Sana"'
image = pipe(
    prompt=prompt,
    height=1024,
    width=1024,
    guidance_scale=4.5,
    num_inference_steps=20,
    generator=torch.Generator(device="cuda").manual_seed(42),
)[0]

image[0].save("sana.png")

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.5.1+cu121)
    Python  3.10.11 (you have 3.10.16)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
A matching Triton is not available, some optimizations will not be enabled
Traceback (most recent call last):
  File "c:\Users\29058\miniconda3\envs\hi\lib\site-packages\xformers\__init__.py", line 57, in _is_triton_available
    import triton  # noqa
ModuleNotFoundError: No module named 'triton'


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OSError: Error no file named config.json found in directory C:\Users\29058\.cache\huggingface\hub\models--Efficient-Large-Model--Sana_Sprint_0.6B_1024px_diffusers\snapshots\a7d9fc31dd5c3f5e22dbfd78360777ceed56ae97\vae.